Name: Buraq Khan

Email: i180800@nu.edu.pk

Roll-num: 18I-0800


In [ ]:
import re

# File Reading

**File Reading And Unigram Generation**





In [ ]:
with open("/content/drive/MyDrive/nlp_a4/data.txt") as f:
  corpus = f.read().split()

vocab = {}
for word in corpus:
  if word not in vocab:
    vocab[word] = 1
  else:
    vocab[word] += 1

unigrams = {}
for key, value in vocab.items():
  unigrams[key] = (value/len(corpus))

# for word in unigrams:
#   print(word, unigrams[word])

In [ ]:
#corpus

**Misspelling Reading**

In [ ]:
with open("/content/drive/MyDrive/nlp_a4/misspellings.txt") as fr:
  misspellings_raw = fr.readlines()

for count, line  in enumerate(misspellings_raw):
  line = re.sub('\s+',' ', line)
  misspellings_raw[count] = line

for count, line in enumerate(misspellings_raw):
  line = line.split()
  misspellings_raw[count] = line

In [ ]:
misspellings = {}
for line in misspellings_raw:
  word = line[0].replace("," , "")
  misspellings[word] = line[1:] 

# for key, value in misspellings.items():
#   print(key, value)

# Table Generation

In [ ]:
del_dict = {}
insert_dict = {}
subs_dict = {}
trans_dict = {}

**Insertion Table**

In [ ]:
def insert(word, error):
  word_list = list(word)
  error_list = list(error)

  for count, char in enumerate(word_list):
    
    #handling special last char case
    if count == len(word_list)-1:
      if error_list[count] == char:
        if (char, error_list[count+1]) in insert_dict.keys():
          insert_dict[(char, error_list[count+1])] += 1
        else:
          insert_dict[(char, error_list[count+1])] = 1
        break
      else:
        if (error_list[count], error_list[count+1]) in insert_dict.keys():
          insert_dict[(error_list[count], error_list[count+1])] += 1
        else:
          insert_dict[(error_list[count], error_list[count+1])] = 1
        break


    if error_list[count] == char:
      continue
    else:
      if count == 0:
        if ("#", error_list[count]) in insert_dict.keys():
          insert_dict[("#", error_list[count])] += 1
        else:
          insert_dict[("#", error_list[count])] = 1
        break
      else:  
        if (error_list[count-1], error_list[count]) in insert_dict.keys():
          insert_dict[(error_list[count-1], error_list[count])] += 1
        else:
          insert_dict[(error_list[count-1], error_list[count])] = 1
        break

**Deletion Table**

In [ ]:
def delete(word, error):
  word_list = list(word)
  error_list = list(error)

  for count, char in enumerate(word_list):
    
    if count == len(word_list)-1:
      if (word_list[count-1], char) in del_dict.keys():
        del_dict[(word_list[count-1], char)] += 1
      else:
        del_dict[(word_list[count-1], char)] = 1
      break
    
    if error_list[count] == char:
      continue
    
    else:
      if count == 0:
        if ("#", char) in del_dict.keys():
          del_dict[("#", char)] += 1
        else:
          del_dict[("#", char)] = 1
        break
      else:
        if (char, error_list[count]) in del_dict.keys():
          del_dict[(char, error_list[count])] += 1
        else:
          del_dict[(char, error_list[count])] = 1
        break

**Substitute and Transpose Tables**

In [ ]:
def subsituteAndTranspose(word, error):
  word_list = list(word)
  error_list = list(error)

  for count, char in enumerate(word_list):
    if error_list[count] == char:
      continue
    else:
      if count < len(word_list)-1:
        if word_list[count+1] == error_list[count] and char == error_list[count+1]:
          if (char,word_list[count+1]) in trans_dict:
            trans_dict[(char, word_list[count+1])] += 1
          else:
            trans_dict[(char, word_list[count+1])] = 1
          break
    
      if (char, error_list[count]) in subs_dict:
        subs_dict[(char, error_list[count])] += 1
      else:
        subs_dict[(char, error_list[count])] = 1
      break

In [ ]:
def findError(word, errors):
  for error in errors:
    if len(error) > len(word):
      insert(word, error)
    elif len(error) < len(word):
      delete(word, error)
    elif len(error) == len(word):
      subsituteAndTranspose(word, error)

In [ ]:
for key, value in misspellings.items():
  word = key
  error_words = value
  findError(word, error_words)

**Channel Model Probability**

In [ ]:
with open("/content/drive/MyDrive/nlp_a4/data.txt") as f:
  test = f.read()

test = re.sub('\s+',' ', test)

In [ ]:
char_dict = {}
for char in test:
  if char is not " ":
    if char in char_dict.keys():
      char_dict[char] += 1
    else:
      char_dict[char] = 1

In [ ]:
char_dict

{'a': 61215602,
 'b': 6203369,
 'c': 2859542,
 'd': 6035947,
 'e': 16187901,
 'f': 1594870,
 'g': 3139263,
 'h': 23399958,
 'i': 21156833,
 'j': 3116738,
 'k': 15094087,
 'l': 6741876,
 'm': 8154672,
 'n': 15830622,
 'o': 10532080,
 'p': 5393392,
 'q': 1426466,
 'r': 15514837,
 's': 11123713,
 't': 10417311,
 'u': 6635599,
 'v': 230208,
 'w': 3549251,
 'x': 32400,
 'y': 6922251,
 'z': 2363619}

In [ ]:
bi_dict = {}
for count, char in enumerate(test):
  if count == len(test)-1:
    break
  else:
    if char is not " " and test[count+1] is not " ":
      if (char, test[count+1]) in bi_dict.keys():
        bi_dict[(char, test[count+1])] += 1
      else:
        bi_dict[(char, test[count+1])] = 1
    elif char == " ":
      if ("#", test[count+1]) in bi_dict.keys():
        bi_dict[("#", test[count+1])] += 1
      else:
        bi_dict[("#", test[count+1])] = 1

In [ ]:
bi_dict

In [ ]:
def calcErrorProbability(char_tuple, mode):
  prob = float('-inf')
  if mode == "delete":
    if char_tuple in del_dict.keys():
      prob = del_dict[char_tuple]/bi_dict[char_tuple]
  elif mode == "insert":
    if char_tuple[0] == "#":
      prob = insert_dict[char_tuple]/len(corpus)
    else:
      if char_tuple in insert_dict.keys():
        prob = insert_dict[char_tuple]/char_dict[char_tuple[0]]
  elif mode == "subsitute":
    if char_tuple in subs_dict.keys():
      prob = subs_dict[char_tuple]/char_dict[char_tuple[0]]
  elif mode == "transpose":
    if char_tuple in trans_dict.keys():
      prob = trans_dict[char_tuple]/bi_dict[char_tuple]

  return prob

In [ ]:
pr = calcErrorProbability(("b","c"), "subsitute")
pr

1.257381271370444e-05

# Candidate Word Generation

**Candidate Words Generated Using Trie Data Structure**


> Trie Data Structure has less space complexity and time complexity: O(n x m)


> Searching done using levenstien's edit distance





In [ ]:
class Trie():
    def __init__(self):
      self.child = {}
      self.word = None

    def insert(self, word):
      node = self
      for letter in word:
        if letter not in node.child:
          node.child[letter] = Trie()
        node = node.child[letter]
      node.word = word

    def search(self, word, cost):
      row = range(len(word) + 1)
      word_list = []
      for char in self.child:
        self.searchCandidates(self.child[char], char, word, row, word_list, cost)
      return word_list

    def searchCandidates(self, node, char, word, prev_row, word_list, cost):
      columns = len(word) + 1
      curr_row = [prev_row[0] + 1]

      for column in range(1, columns):
        insert = curr_row[column - 1] + 1
        delete = prev_row[column] + 1
        if word[column - 1] != char:
            replace = prev_row[column - 1] + 1
        else:
            replace = prev_row[column - 1]

        curr_row.append(min(insert, delete, replace))

      if curr_row[-1] <= cost and node.word is not None:
        word_list.append((node.word, curr_row[-1]))

      if min(curr_row) <= cost:
        for char in node.child:
          self.searchCandidates(node.child[char], char, word, curr_row, word_list, cost)

def findTranspose(w_list, w):
  trans_list = []
  for tup in w_list:
    check = 0
    word = tup[0]
    e_dist = tup[1]
    if len(word) == len(w) and e_dist == 2:
      for count, char in enumerate(word):
        if char is not w[count]:
          if char == w[count+1] and word[count+1] == w[count]:
            if word[count+2:] == w[count+2:]:
              trans_list.append(tup[0])
          break
  return trans_list

In [ ]:
trie = Trie()
for word in unigrams.keys():
  trie.insert(word)

In [ ]:
def findCandidateWords(misspell_word):
  candidate_words = []
  test_list = []
  word_list = trie.search(misspell_word, 2)
  for val in word_list:
    if val[1] == 1:
      candidate_words.append(val[0])
  test_list = findTranspose(word_list, misspell_word)
  candidate_words.extend(test_list)
  
  return candidate_words

In [ ]:
words = findCandidateWords("hello")
words

['hell', 'helly', 'ello']

# Most Probable Word Generation

In [ ]:
def checkDelete(correct_word, miss_word):
  for count, char in enumerate(miss_word):
    if count == len(miss_word)-1:
      return (char, correct_word[count+1])
    else:
      if char == correct_word[count]:
        continue
      else:
        if count == 0:
          return ("#", correct_word[count])
        else:
          return (correct_word[count-1], correct_word[count])

def checkInsert(correct_word, miss_word):
  for count, char in enumerate(miss_word):
    if count == len(miss_word)-1:
      return (miss_word[count-1], char)
    else:
      if char == correct_word[count]:
        continue
      else:
        if count == 0:
          return ("#", char)
        else:
          return (miss_word[count-1], char)

def checkSubsitute(correct_word, miss_word):
  for count, char in enumerate(miss_word):
    if char == correct_word[count]:
      continue
    else:
      if count != len(miss_word)-1:
        if char == correct_word[count+1] and correct_word[count] == miss_word[count+1]:
          return None
      return (correct_word[count], char)

def checkTranspose(correct_word, miss_word):
  for count, char in enumerate(miss_word):
    if char == correct_word[count]:
      continue
    else:
      if char == correct_word[count+1] and correct_word[count] == miss_word[count+1]:
        return (correct_word[count], char)

In [ ]:
def findFunction(correct_word, miss_word):
  # Deletion
  if len(miss_word) < len(correct_word):
    return "delete", checkDelete(correct_word, miss_word)  
  
  # Insertion
  elif len(miss_word) > len(correct_word):
    return "insert", checkInsert(correct_word, miss_word)
  
  # Subsitute Or Transpose
  elif len(miss_word) == len(correct_word):
    if checkSubsitute(correct_word, miss_word):
      return "subsitute", checkSubsitute(correct_word, miss_word)
    else:
      return "transpose", checkTranspose(correct_word, miss_word)

**Finding most probable word**

In [ ]:
def spellCorrection(miss_word):
  word_dict = dict()
  candidate_words = findCandidateWords(miss_word)
  if len(candidate_words) == 0:
    return miss_word
  else:
    for correct_word in candidate_words:
      mode, char_tuple = findFunction(correct_word, miss_word)
      bayes_prob = calcErrorProbability(char_tuple, mode) * unigrams[correct_word]    # Equation: P(w|x) . P(w)
      word_dict[correct_word] = bayes_prob

    correct_guess = max(word_dict, key = word_dict.get)   # Arg Max from the dictionary of words with their probabilities
  
  print(word_dict)
  return correct_guess

In [ ]:
spellCorrection("hello")

{'hell': 6.429668860835294e-11, 'helly': 2.411875342086612e-10, 'ello': 5.41310545710595e-11}


'helly'